In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Union, List
import metadsl
import metadsl_rewrite
from metadsl_core.strategies import register_core

In [2]:
@dataclass
class Expression(metadsl.Expression):
    """Base expression."""


class TableBase(Expression):
    def __getitem__(
        self, key: Union[str, List[str]]
    ) -> Union[Projection, Column]:
        """Get item from Table."""
        if isinstance(key, str):
            expr = self.column(key)
        else:
            expr = self.projection(key)
        # expr.__qualname__ = 'Table'
        # expr.__name__ = 'Table'
        return expr
        
    @metadsl.expression
    def projection(self, keys: list) -> Projection:
        """Create a projection expression."""

    @metadsl.expression
    def column(self, key: str) -> Column:
        """Create a column expression."""


class Table(TableBase):
    """Table base expression class."""


class Projection(TableBase):
    """Projection expression."""
    

class Column(TableBase):
    """Column expression"""



In [3]:
@metadsl.expression
def table(data: dict) -> Table:
    """Create a table expression."""


In [4]:
expr = table({'a': [1, 2, 3]})

In [5]:
type(expr)

__main__.Table

In [6]:
expr['a']

Column(function=Table.column, args=[Table(function=table, args=[{'a': [1, 2, 3]}], kwargs={}), 'a'], kwargs={})

In [7]:
expr[['a']]

Projection(function=Table.projection, args=[Table(function=table, args=[{'a': [1, 2, 3]}], kwargs={}), ['a']], kwargs={})